# Shareholder Service - Ownership Structure Analysis

Track major shareholders, free float percentages, and ownership distribution for investment analysis.

## Overview

The **Shareholder Service** provides ownership data:

- **Major Shareholders**: Top shareholders with ownership percentages
- **Free Float**: Tradable shares percentage and holder count
- **NVDR Identification**: Non-Voting Depositary Receipt holders
- **Ownership Concentration**: Analyze control and liquidity
- **Foreign vs Local**: Track nationality of major holders

### When to Use

- Liquidity analysis (free float monitoring)
- Control analysis (ownership concentration)
- Corporate governance research
- M&A potential assessment
- Shareholder activism tracking

In [1]:
!pip install settfex

zsh:1: command not found: pip


In [2]:
import asyncio
from settfex.services.set import get_shareholder_data
import pandas as pd

print("Libraries imported!")

Libraries imported!


## Basic Usage

In [ ]:
# Fetch shareholder data
data = await get_shareholder_data("MINT")

print(f"Symbol: {data.symbol}")
print(f"Total Shareholders: {data.total_shareholder:,}")
print(f"Book Close Date: {data.book_close_date}")
print(f"\nFree Float:")
print(f"  Percentage: {data.free_float.percent_free_float:.2f}%")
print(f"  Number of Holders: {data.free_float.number_of_holder:,}")
print(f"\nTop 10 Major Shareholders:\n")

for sh in data.major_shareholders[:10]:
    nvdr_flag = " [NVDR]" if sh.is_thai_nvdr else ""
    print(f"{sh.sequence}. {sh.name}{nvdr_flag}")
    print(f"   Nationality: {sh.nationality}")
    print(f"   Shares: {sh.number_of_share:,} ({sh.percent_of_share:.2f}%)")
    print()

## Advanced Usage - Ownership Concentration

In [ ]:
async def analyze_ownership_concentration(symbols: list[str]):
    """
    Analyze ownership concentration for multiple stocks.
    """
    tasks = [get_shareholder_data(symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    analysis = []
    for symbol, data in zip(symbols, results):
        if isinstance(data, Exception):
            continue
        
        # Calculate concentration metrics
        top5_pct = sum(sh.percent_of_share for sh in data.major_shareholders[:5])
        top10_pct = sum(sh.percent_of_share for sh in data.major_shareholders[:10])
        
        analysis.append({
            "symbol": symbol,
            "total_shareholders": data.total_shareholder,
            "free_float_pct": data.free_float.percent_free_float,
            "top5_ownership": top5_pct,
            "top10_ownership": top10_pct,
            "major_shareholders_count": len(data.major_shareholders)
        })
    
    df = pd.DataFrame(analysis).sort_values('top5_ownership', ascending=False)
    return df

# Analyze blue chips
symbols = ["PTT", "KBANK", "CPALL", "AOT", "BBL", "MINT"]
concentration = await analyze_ownership_concentration(symbols)

print("Ownership Concentration Analysis:\n")
print(concentration.to_string(index=False))

## Use Case: Foreign vs Local Shareholders

In [ ]:
# Analyze nationality distribution
data = await get_shareholder_data("CPALL")

thai_holders = [sh for sh in data.major_shareholders if sh.nationality == "THAI"]
foreign_holders = [sh for sh in data.major_shareholders if sh.nationality != "THAI"]

thai_pct = sum(sh.percent_of_share for sh in thai_holders)
foreign_pct = sum(sh.percent_of_share for sh in foreign_holders)

print(f"Ownership by Nationality for {data.symbol}:")
print(f"\nThai Shareholders: {len(thai_holders)} ({thai_pct:.2f}%)")
print(f"Foreign Shareholders: {len(foreign_holders)} ({foreign_pct:.2f}%)")
print(f"\nTop Foreign Shareholders:")
for sh in foreign_holders[:5]:
    print(f"  {sh.name} ({sh.nationality}): {sh.percent_of_share:.2f}%")

## Use Case: NVDR Analysis

In [ ]:
# Identify NVDR shareholders
nvdr_holders = [sh for sh in data.major_shareholders if sh.is_thai_nvdr]

print(f"NVDR Shareholders: {len(nvdr_holders)}\n")
for sh in nvdr_holders:
    print(f"{sh.name}")
    print(f"  Ownership: {sh.percent_of_share:.2f}%")
    print(f"  Shares: {sh.number_of_share:,}")
    print()

## Next Steps

- **[NVDR Holder](07_nvdr_holder.ipynb)** - NVDR-specific data
- **[Company Profile](04_company_profile.ipynb)** - Governance and management
- **[Stock Profile](03_stock_profile.ipynb)** - Foreign ownership limits

**Documentation**: `/docs/settfex/services/set/shareholder.md`